<a href="https://colab.research.google.com/github/beiyouwuyanzu/llama_application/blob/main/Copy_of_Chinese_Vicuna_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown "https://drive.google.com/uc?id=1tzXVhS74m-EtoFot7hEc005LDeZGPit_" -O merge.json

Downloading...
From: https://drive.google.com/uc?id=1tzXVhS74m-EtoFot7hEc005LDeZGPit_
To: /content/merge.json
100% 696M/696M [00:03<00:00, 195MB/s]


In [ ]:
!git clone https://github.com/Facico/Chinese-Vicuna
!pip install -r ./Chinese-Vicuna/requirements.txt

In [ ]:
!python ./Chinese-Vicuna/finetune.py --data_path merge.json --test_size 2000

In [ ]:
import os
import sys

import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset
import transformers
import argparse
import warnings


assert (
    "LlamaTokenizer" in transformers._import_structure["models.llama"]
), "LLaMA is now in HuggingFace's main branch.\nPlease reinstall it: pip uninstall transformers && pip install git+https://github.com/huggingface/transformers.git"
from transformers import LlamaForCausalLM, LlamaTokenizer
from peft import (
    prepare_model_for_int8_training,
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
)

In [4]:
DATA_PATH = "./merge.json" #"/home/cciip/private/fanchenghao/dataset/instruction/merge.json"
OUTPUT_DIR = "./output" #"lora-Vicuna"

device_map = "auto"
world_size = int(os.environ.get("WORLD_SIZE", 1))

In [ ]:
model_path = "decapoda-research/llama-7b-hf"

ddp = world_size != 1
if ddp:
    device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)}
    GRADIENT_ACCUMULATION_STEPS = GRADIENT_ACCUMULATION_STEPS // world_size
print(model_path)
model = LlamaForCausalLM.from_pretrained(
    model_path,
    load_in_8bit=True,
    device_map=device_map,
)

In [8]:
tokenizer = LlamaTokenizer.from_pretrained(
    model_path, add_eos_token=True
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [11]:
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.05
TARGET_MODULES = [
    "q_proj",
    "v_proj",
]

config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)

In [12]:
model = get_peft_model(model, config)

In [13]:
tokenizer.pad_token_id = 0

In [14]:
VAL_SET_SIZEdata = load_dataset("json", data_files=DATA_PATH)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-9a2b36bbad088d0b/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
VAL_SET_SIZE = 2000
MICRO_BATCH_SIZE = 4  # this could actually be 5 but i like powers of 2
BATCH_SIZE = 128
EPOCHS = 3  # we don't always need 3 tbh
now_max_steps = max((len(data["train"]) - VAL_SET_SIZE) // BATCH_SIZE * EPOCHS, EPOCHS)

In [19]:
print(now_max_steps)

16218


In [20]:
MAX_STEPS = now_max_steps

In [21]:
model.print_trainable_parameters()

trainable params: 4194304 || all params: 6742609920 || trainable%: 0.06220594176090199


In [22]:
def generate_prompt(data_point):
    # sorry about the formatting disaster gotta move fast
    if data_point["input"]:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{data_point["instruction"]}

### Input:
{data_point["input"]}

### Response:
{data_point["output"]}"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{data_point["instruction"]}

### Response:
{data_point["output"]}"""

In [23]:
def tokenize(prompt):
    # there's probably a way to do this with the tokenizer settings
    # but again, gotta move fast
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN + 1,
        padding="max_length",
    )
    return {
        "input_ids": result["input_ids"][:-1],
        "attention_mask": result["attention_mask"][:-1],
    }


In [24]:
def generate_and_tokenize_prompt(data_point):
    # This function masks out the labels for the input,
    # so that our loss is computed only on the response.
    user_prompt = (
        (
            f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{data_point["instruction"]}

### Input:
{data_point["input"]}

### Response:
"""
        )
        if data_point["input"]
        else (
            f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{data_point["instruction"]}

### Response:
"""
        )
    )
    len_user_prompt_tokens = (
        len(
            tokenizer(
                user_prompt,
                truncation=True,
                max_length=CUTOFF_LEN + 1,
            )["input_ids"]
        )
        - 1
    )  # no eos token
    full_tokens = tokenizer(
        user_prompt + data_point["output"],
        truncation=True,
        max_length=CUTOFF_LEN + 1,
        padding="max_length",
    )["input_ids"][:-1]
    return {
        "input_ids": full_tokens,
        "labels": [-100] * len_user_prompt_tokens
        + full_tokens[len_user_prompt_tokens:],
        "attention_mask": [1] * (len(full_tokens)),
    }



In [25]:
print(VAL_SET_SIZE)

2000


In [27]:
LEARNING_RATE = 3e-4  # the Karpathy constant
CUTOFF_LEN = 256  # 256 accounts for about 96% of the data
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.05


train_val = data["train"].train_test_split(
    test_size=VAL_SET_SIZE, shuffle=True, seed=42
)
train_data = train_val["train"].shuffle().map(generate_and_tokenize_prompt)
val_data = train_val["test"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/691987 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [28]:
print(train_data)

Dataset({
    features: ['instruction', 'input', 'output', 'input_ids', 'labels', 'attention_mask'],
    num_rows: 691987
})


In [ ]:
val_data[0]

In [30]:
example = val_data[0]

In [31]:
print(example)

{'instruction': '给我推荐一些适合夏天穿的款式时尚衣服。\\n\n', 'input': '', 'output': '1. 连体短裤/连衣裙\n2. 露肩上衣/连衣裙\n3. 宽松T恤\n4. 花卉印花短裙\n5. 纯棉针织背心\n6. 薄棉质长裤\n7. 无袖连衣裙\n8. 男/女运动套装\n9. 水蓝色或白色短裤/短裙\n10. 宽边草帽和太阳镜', 'input_ids': [0, 13866, 338, 385, 15278, 393, 16612, 263, 3414, 29889, 14350, 263, 2933, 393, 7128, 2486, 1614, 2167, 278, 2009, 29889, 13, 13, 2277, 29937, 2799, 4080, 29901, 13, 31999, 30672, 233, 145, 171, 235, 144, 147, 30287, 31959, 236, 131, 133, 30733, 31241, 30408, 234, 172, 194, 30210, 233, 175, 193, 30607, 30594, 232, 179, 157, 235, 164, 166, 31520, 30267, 29905, 29876, 13, 13, 13, 2277, 29937, 13291, 29901, 13, 29896, 29889, 29871, 31903, 30988, 234, 162, 176, 235, 166, 167, 29914, 31903, 235, 164, 166, 235, 166, 156, 13, 29906, 29889, 29871, 236, 159, 181, 235, 133, 172, 30429, 235, 164, 166, 29914, 31903, 235, 164, 166, 235, 166, 156, 13, 29941, 29889, 29871, 232, 177, 192, 31018, 29911, 233, 132, 167, 13, 29946, 29889, 29871, 30830, 232, 144, 140, 232, 144, 179, 30830, 234, 162, 176, 235

In [32]:
print(example.keys())

dict_keys(['instruction', 'input', 'output', 'input_ids', 'labels', 'attention_mask'])


In [35]:
for k, v in example.items():
  print(k, len(v), v)

instruction 22 给我推荐一些适合夏天穿的款式时尚衣服。\n

input 0 
output 108 1. 连体短裤/连衣裙
2. 露肩上衣/连衣裙
3. 宽松T恤
4. 花卉印花短裙
5. 纯棉针织背心
6. 薄棉质长裤
7. 无袖连衣裙
8. 男/女运动套装
9. 水蓝色或白色短裤/短裙
10. 宽边草帽和太阳镜
input_ids 256 [0, 13866, 338, 385, 15278, 393, 16612, 263, 3414, 29889, 14350, 263, 2933, 393, 7128, 2486, 1614, 2167, 278, 2009, 29889, 13, 13, 2277, 29937, 2799, 4080, 29901, 13, 31999, 30672, 233, 145, 171, 235, 144, 147, 30287, 31959, 236, 131, 133, 30733, 31241, 30408, 234, 172, 194, 30210, 233, 175, 193, 30607, 30594, 232, 179, 157, 235, 164, 166, 31520, 30267, 29905, 29876, 13, 13, 13, 2277, 29937, 13291, 29901, 13, 29896, 29889, 29871, 31903, 30988, 234, 162, 176, 235, 166, 167, 29914, 31903, 235, 164, 166, 235, 166, 156, 13, 29906, 29889, 29871, 236, 159, 181, 235, 133, 172, 30429, 235, 164, 166, 29914, 31903, 235, 164, 166, 235, 166, 156, 13, 29941, 29889, 29871, 232, 177, 192, 31018, 29911, 233, 132, 167, 13, 29946, 29889, 29871, 30830, 232, 144, 140, 232, 144, 179, 30830, 234, 162, 176, 235, 166, 156, 13, 2994